In [0]:
import delta

In [0]:
df_full = spark.read.format("csv").options(sep=";", header=True).load("/Volumes/workspace/upsell/full_load/customers/")
(df_full.coalesce(1).write.format("delta").mode("overwrite").saveAsTable("bronze.customers"))

### Atualização da tabela - Incremental

In [0]:
# Cria uma view para executar consultas SQL.
(spark.read
  .format("csv")
  .options(sep=";", header=True)
  .load("/Volumes/workspace/upsell/cdc/customers/")
  .createOrReplaceTempView("customers"))

# Consulta para pegar o dado mais recente de cada cliente para realizar o incremental.
query = '''
    SELECT * 
    FROM customers
    QUALIFY ROW_NUMBER() OVER (PARTITION BY idCliente ORDER BY DtAtualizacao DESC) = 1
'''

df_cdc_unique = spark.sql(query)

In [0]:
bronze = delta.DeltaTable.forName(spark, "bronze.customers")

# Consolidação do dado mais recente entre a Bronze e os novos dados que chegou, denominado UPSERT.
(bronze.alias("b")
       .merge(df_cdc_unique.alias("d"), "b.idCliente = d.idCliente")
       .whenMatchedDelete(condition = "d.OP = 'D'")
       .whenMatchedUpdateAll(condition = "d.OP = 'U'")
       .whenNotMatchedInsertAll(condition = "d.OP = 'I' or d.OP = 'U'")
       #.execute()
)